In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone https://github.com/totalgood/nlpia.git

Cloning into 'nlpia'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 5804 (delta 84), reused 118 (delta 49), pack-reused 5631
Receiving objects: 100% (5804/5804), 124.00 MiB | 27.47 MiB/s, done.
Resolving deltas: 100% (3660/3660), done.


In [0]:
df = pd.read_csv('/content/nlpia/src/nlpia/data/hutto_ICWSM_2014/amazonReviewSnippets_GroundTruth.txt', delimiter = '\t')

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3546 entries, 0 to 3545
Data columns (total 3 columns):
id           3546 non-null object
sentiment    3546 non-null float64
text         3546 non-null object
dtypes: float64(1), object(2)
memory usage: 83.2+ KB


In [0]:
df.head()

,id,sentiment,text
0,1_1,-0.90,troubleshooting ad-2500 and ad-2600 no picture...
1,1_2,-0.15,"repost from january 13, 2004 with a better fit..."
2,1_3,-0.20,does your apex dvd player only play dvd audio ...
3,1_4,-0.10,or does it play audio and video but scrolling ...
4,1_5,-0.50,before you try to return the player or waste h...


In [0]:
df.shape

(3546, 3)

In [0]:
df.describe().round(2)

,sentiment
count,3546.00
mean,0.40
std,1.63
min,-3.70
25%,-0.95
50%,0.35
75%,1.80
max,3.85


In [0]:
pd.set_option('display.width', 75)
from nltk.tokenize import casual_tokenize
bags_of_words = []
from collections import Counter
for text in df.text:
  bags_of_words.append(Counter(casual_tokenize(text)))
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape

(3546, 5687)

In [0]:
df_bows.head()

,!,"""",#,#38,$,%,&,',(,(8,),):,);,*,+,",",-,-0.05,-0.15,-0.25,-0.35,-0.4,-0.45,-0.55,-0.7,-0.75,-0.8,-0.9,-0.95,-1.15,-1.2,-1.25,-1.3,-1.35,-1.4,-1.45,-1.5,-1.55,-1.6,-1.65,...,ya,yeah,year,years,yells,yep,yes,yesterday,yet,yield,yielded,yo,you,you'll,you're,you've,youd,your,youre,yours,yourself,yoursound,youve,yup,zeiss,zen,zen's,zenith,zennx,zennx's,zens,zero,zillion,zip,zoom,zoomed,zooming,zooms,zx,~
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_bows.head()[list(bags_of_words)[0].keys()]

,troubleshooting,ad,-,2500,and,2600,no,picture,scrolling,b,/,w,.
0,1,2,2,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb = nb.fit(df_bows, df.sentiment>0)

In [0]:
df['predicted_sentiment'] = nb.predict_proba(df_bows)[:,1] * 8 - 4

In [0]:
df.describe()

,sentiment,predicted_sentiment,error,sentiment_ispositive,predicted_ispositive
count,3546.000000,3546.000000,3546.000000,3546.000000,3546.000000
mean,0.403511,0.804369,2.012433,0.578116,0.613367
std,1.625920,3.212701,1.039449,0.493930,0.487047
min,-3.700000,-4.000000,0.000175,0.000000,0.000000
25%,-0.950000,-2.960999,1.249461,0.000000,0.000000
50%,0.350000,2.225377,1.977846,1.000000,1.000000
75%,1.800000,3.878308,2.736372,1.000000,1.000000
max,3.850000,4.000000,6.577115,1.000000,1.000000


In [0]:
df['error'] = (df['predicted_sentiment'] - df['sentiment']).abs()

In [0]:
df['sentiment_ispositive'] = (df['sentiment'] > 0).astype(int)

df['predicted_ispositive'] = (df['predicted_sentiment'] > 0).astype(int)

In [0]:
df.head(6)

,id,sentiment,text,predicted_sentiment,error,sentiment_ispositive,predicted_ispositive
0,1_1,-0.90,troubleshooting ad-2500 and ad-2600 no picture...,-3.905995,3.005995,0,0
1,1_2,-0.15,"repost from january 13, 2004 with a better fit...",-3.839207,3.689207,0,0
2,1_3,-0.20,does your apex dvd player only play dvd audio ...,-3.913626,3.713626,0,0
3,1_4,-0.10,or does it play audio and video but scrolling ...,-3.991184,3.891184,0,0
4,1_5,-0.50,before you try to return the player or waste h...,-3.999056,3.499056,0,0
5,1_6,-0.95,no picture: hopefully you still have the remo...,0.549298,1.499298,0,1


In [0]:
(df['sentiment_ispositive']==df['predicted_ispositive']).sum()/len(df)

0.8846587704455725

In [0]:
product_df = pd.read_csv('/content/nlpia/src/nlpia/data/hutto_ICWSM_2014/tweets_GroundTruth.txt', delimiter = '\t')

In [0]:
product_df.head()

,id,sentiment,text,sentiment_ispositive
0,1,2.726316,Somehow I was blessed with some really amazing...,1
1,2,1.443299,Yay. Another good phone interview.,1
2,3,2.873684,We were 17 deep last night &amp; the love was ...,1
3,4,2.857143,"LMAO, AMAZING!",1
4,5,-2.154639,Two words that should die this year: Sexting a...,0


In [0]:
product_df['sentiment_ispositive'] = (product_df['sentiment'] > 0).astype(int)

In [0]:
product_bow = []
for t in product_df.text:
  product_bow.append(Counter(casual_tokenize(t)))
p_df = pd.DataFrame.from_records(product_bow)
p_df = p_df.fillna(0).astype(int)

In [0]:
df_bows.head()

,!,"""",#,#38,$,%,&,',(,(8,),):,);,*,+,",",-,-0.05,-0.15,-0.25,-0.35,-0.4,-0.45,-0.55,-0.7,-0.75,-0.8,-0.9,-0.95,-1.15,-1.2,-1.25,-1.3,-1.35,-1.4,-1.45,-1.5,-1.55,-1.6,-1.65,...,ya,yeah,year,years,yells,yep,yes,yesterday,yet,yield,yielded,yo,you,you'll,you're,you've,youd,your,youre,yours,yourself,yoursound,youve,yup,zeiss,zen,zen's,zenith,zennx,zennx's,zens,zero,zillion,zip,zoom,zoomed,zooming,zooms,zx,~
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
p_df.head()

,!,"""",#,#100words,#12,#13thingsilove,#2ndGen,#39,#6WordStory,#6nations,#ACC,#AnIdiotAbroad,#Apple,#AppleCare,#AppleStore,#Athletics,#Auburn,#BB13,#BOLTUP,#BananaPhobia,#Bangalore,#BeSimple,#BecauseYoureChosen,#BlackIsBack,#Boehner,#BriansDailyThanks,#BridalShower,#Bride,#Bucs,#CDWM,#CES,#CSS,#Capricorn,#CatchingHell,#ChildFree,#Comedy,#Cowboys,#CurtisGrimes,#CyberMonday,#Cypher,...,yfrog,yo,yo-yo's,yoga,yoooo,yoplait,york,you,you'd,you'll,you're,you've,youTube,young,younger,your,youre,yours,yoursel,yourself,yousuf,youth,youtube,yr,yrs,yummy,yung,yup,zamanda,zebra,zero,zipper,zombie-like,zumba,zzzzzzz,{,|,},~,£
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_all_bows = df_bows.append(p_df)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [0]:
df_all_bows = df_all_bows.fillna(0).astype(int)

In [0]:
df_all_bows.head()

,!,"""",#,#100words,#12,#13thingsilove,#2ndGen,#38,#39,#6WordStory,#6nations,#ACC,#AnIdiotAbroad,#Apple,#AppleCare,#AppleStore,#Athletics,#Auburn,#BB13,#BOLTUP,#BananaPhobia,#Bangalore,#BeSimple,#BecauseYoureChosen,#BlackIsBack,#Boehner,#BriansDailyThanks,#BridalShower,#Bride,#Bucs,#CDWM,#CES,#CSS,#Capricorn,#CatchingHell,#ChildFree,#Comedy,#Cowboys,#CurtisGrimes,#CyberMonday,...,youre,yours,yoursel,yourself,yoursound,yousuf,youth,youtube,youve,yr,yrs,yummy,yung,yup,zamanda,zebra,zeiss,zen,zen's,zenith,zennx,zennx's,zens,zero,zillion,zip,zipper,zombie-like,zoom,zoomed,zooming,zooms,zumba,zx,zzzzzzz,{,|,},~,£
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_product_bows = df_all_bows.iloc[len(df):][df_bows.columns]

In [0]:
df_product_bows.head()

,!,"""",#,#38,$,%,&,',(,(8,),):,);,*,+,",",-,-0.05,-0.15,-0.25,-0.35,-0.4,-0.45,-0.55,-0.7,-0.75,-0.8,-0.9,-0.95,-1.15,-1.2,-1.25,-1.3,-1.35,-1.4,-1.45,-1.5,-1.55,-1.6,-1.65,...,ya,yeah,year,years,yells,yep,yes,yesterday,yet,yield,yielded,yo,you,you'll,you're,you've,youd,your,youre,yours,yourself,yoursound,youve,yup,zeiss,zen,zen's,zenith,zennx,zennx's,zens,zero,zillion,zip,zoom,zoomed,zooming,zooms,zx,~
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df_product_bows.shape

(4200, 5687)

In [0]:
product_df.head()

,id,sentiment,text,sentiment_ispositive
0,1,2.726316,Somehow I was blessed with some really amazing...,1
1,2,1.443299,Yay. Another good phone interview.,1
2,3,2.873684,We were 17 deep last night &amp; the love was ...,1
3,4,2.857143,"LMAO, AMAZING!",1
4,5,-2.154639,Two words that should die this year: Sexting a...,0


In [0]:
product_df['predicted_ispositive'] = nb.predict(df_product_bows).astype(int)

In [0]:
product_df.head()

,id,sentiment,text,sentiment_ispositive,predicted_ispositive
0,1,2.726316,Somehow I was blessed with some really amazing...,1,1
1,2,1.443299,Yay. Another good phone interview.,1,1
2,3,2.873684,We were 17 deep last night &amp; the love was ...,1,1
3,4,2.857143,"LMAO, AMAZING!",1,1
4,5,-2.154639,Two words that should die this year: Sexting a...,0,0


In [0]:
(product_df['sentiment_ispositive'] == product_df['predicted_ispositive']).sum()/len(product_df)

0.6309523809523809